# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [9]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

In [12]:
from azureml.train.automl import AutoMLConfig

In [21]:
from azureml.core import Model

In [29]:
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import Webservice, AciWebservice

In [30]:
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
```
In this prooject I am using heart failure clinical records dataset from UCI machine learning repository which follow ups death events based on 12 clinical features.
```
TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

`UCI ML Repository for this dataset is` - [link](https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv)

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-azure-capstone-project'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
print(ws)

Workspace name: quick-starts-ws-143110
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-143110
Workspace.create(name='quick-starts-ws-143110', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-143110')


In [8]:
compute_cluster_name = "Saturn-AI"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
except ComputeTargetException:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)

compute_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
DATASET_URI = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'

In [11]:
key = 'Heart Failure Clinical Records'

if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset exists in the datastore.")

else:
    dataset = Dataset.Tabular.from_delimited_files(DATASET_URI)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
print(f'Dataframe shape:', df.shape)

train, test = train_test_split(df, test_size=.33, shuffle=True)
print(f'Train count: ', len(train))
print(f'Test count: ', len(test))
train.to_csv('train.csv',index = False)
test.to_csv('test.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])
datastore.upload_files(files = ['./test.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])
test = Dataset.Tabular.from_delimited_files([(datastore,'test.csv')])

Dataset exists in the datastore.
Dataframe shape: (299, 13)
Train count:  200
Test count:  99
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files
Uploading an estimated of 1 files
Target already exists. Skipping upload for test.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
# TODO: Put your automl settings here
automl_settings = {} ## Defined in altogether with config

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_cluster,
    task="classification",
    training_data=train,
    label_column_name="DEATH_EVENT",
    primary_metric="accuracy",
    n_cross_validations=5,
    experiment_timeout_minutes=30,
    enable_early_stopping=True,
    max_concurrent_iterations=4)

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on Saturn-AI with default configuration
Running on remote compute: Saturn-AI


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-azure-capstone-project,AutoML_7dfd39c3-7a02-4dea-bd45-57388d5a593f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-azure-capstone-project,AutoML_7dfd39c3-7a02-4dea-bd45-57388d5a593f,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_7dfd39c3-7a02-4dea-bd45-57388d5a593f',
 'target': 'Saturn-AI',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T06:11:29.834521Z',
 'endTimeUtc': '2021-04-19T06:39:14.787395Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'Saturn-AI',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-azure-capstone-project","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-143110","workspace_name":"quick-starts-ws-143110","region":"southcentralus","compute_target":"Saturn-AI","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validatio

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
best_run, model = remote_run.get_output()
best_run_tags = best_run.get_tags()
best_run_metrics = best_run.get_metrics()

WARNING - The model you attempted to retrieve requires 'azureml-train-automl-runtime' to be installed at '==1.25.0'. Please install 'azureml-train-automl-runtime==1.25.0' (e.g. `pip install azureml-train-automl-runtime==1.25.0`) and then rerun the previous command.


In [19]:
best_run_tags

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":4,"CurrentNodeCount":4}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 'ensembled_iterations': '[28, 31, 39, 36, 19, 10]',
 'ensembled_algorithms': "['LightGBM', 'LightGBM', 'XGBoostClassifier', 'LightGBM', 'LightGBM', 'XGBoostClassifier']",
 'ensemble_weights': '[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]',
 'best_individual_pipeline_score': '0.8799999999999999',
 'best_individual_iteration': '28',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_7dfd39c3-7a02-4dea-bd45-57388d5a593f_ModelExplain',
 'model_explanation': 'True'}

In [20]:
best_run_metrics

{'average_precision_score_macro': 0.8787650500709356,
 'f1_score_weighted': 0.8773029792285054,
 'log_loss': 0.3648240126044199,
 'recall_score_weighted': 0.885,
 'recall_score_micro': 0.885,
 'balanced_accuracy': 0.8036260822510822,
 'AUC_macro': 0.9089930633002516,
 'norm_macro_recall': 0.6072521645021645,
 'accuracy': 0.885,
 'precision_score_weighted': 0.884593578694919,
 'recall_score_macro': 0.8036260822510822,
 'f1_score_micro': 0.885,
 'matthews_correlation': 0.6796509440798123,
 'AUC_micro': 0.9263750000000002,
 'precision_score_micro': 0.885,
 'precision_score_macro': 0.8832279938857264,
 'average_precision_score_weighted': 0.9208642878664453,
 'f1_score_macro': 0.8293225060554272,
 'average_precision_score_micro': 0.9278879094270721,
 'weighted_accuracy': 0.9308981279592097,
 'AUC_weighted': 0.9089930633002519,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_7dfd39c3-7a02-4dea-bd45-57388d5a593f_46/accuracy_table',
 'confusion_matrix': 'aml://artifactId/Experim

In [25]:
joblib.dump(model,'models/automl.pkl')

['models/automl.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [28]:
best_run.register_model(model_path='outputs/model.pkl', model_name='automl',
                        tags={'Name':'AutoML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

Model(workspace=Workspace.create(name='quick-starts-ws-143110', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-143110'), name=automl, id=automl:1, version=1, tags={'Name': 'AutoML'}, properties={'AUC': '0.9089930633002519', 'Accuracy': '0.885'})

In [38]:
best_run.download_file(constants.CONDA_ENV_FILE_PATH, "env.yml")

In [41]:
env  = os.getenv("env.yml")
env

In [40]:
inference_config = InferenceConfig(entry_script="scoring.py", environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [ ]:
service = Model.deploy(workspace=ws,
                      name="automl-service",
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aci_config,
                      overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service